In [1]:
from pyquil import get_qc, Program
from pyquil.gates import *
from pyquil.api import local_forest_runtime
from pyquil.quilbase import Declare
from pyquil.simulation.tools import lifted_gate, program_unitary
from pyquil.quil import *

In [2]:
import numpy as np
import math
from math import pi
import random
import copy

In [3]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas() #This line make sure that the progress bars looks naturalfrom tqdm import tqdm_notebook as tqdm

In [4]:
from functions import *

In [5]:
# # %%writefile -a functions.py

# def native_universal_two_qubits_packs_generator(qmachine, target_qubits:list, num_layer:int):
#     list_gates = []
#     for index in range(num_layer):
#         draft_circuit = give_random_two_qubit_circuit(target_qubits)
#         list_gates.extend( qmachine.compiler.quil_to_native_quil(draft_circuit) )
#     list_gates = [ ins for ins in list_gates if isinstance(ins, Gate)]
#     list_gates.extend( get_inverse_circuit(qmachine, list_gates) )
#     return list_gates

# def native_rigetti_single_qubit_packs_generator(qmachine, target_qubit, num_layer:int):
#     try:
#         temp = iter(target_qubit)
#         if len(target_qubit) == 1:
#             target_qubit = target_qubit[0]
#         else:
#             raise ValueError('target qubit should be only one index')
#     except:
#         pass
    
#     list_gates = []
#     angles = np.linspace(0, np.pi, 100)
    
#     for index in range(0,num_layer):
#         omega, phi = np.random.uniform(0, 2*np.pi, size = 2)
#         theta = np.random.choice(angles, p = np.sin(angles) / np.sum( np.sin(angles) ))
        
#         draft_circuit = Program( [RZ(phi, qubit = target_qubit),
#                                   RY(theta, qubit = target_qubit),
#                                   RZ(omega, qubit = target_qubit)])
        
#         list_gates.extend(qmachine.compiler.quil_to_native_quil(draft_circuit))
    
#     list_gates = [ ins for ins in list_gates if isinstance(ins, Gate)]
#     list_gates.extend( get_inverse_circuit(qmachine, list_gates) )
#     return list_gates

In [6]:
# def clifford_group_packs_generator(qmachine, target_qubit, num_layer:int):
    
#     list_gates = []
    
#     for index in range(0,num_layer):
#         omega, phi = np.random.uniform(0, 2*np.pi, size = 2)
#         theta = np.random.choice(angles, p = np.sin(angles) / np.sum( np.sin(angles) ))
        
#         draft_circuit = Program( [RZ(phi, qubit = target_qubit),
#                                   RY(theta, qubit = target_qubit),
#                                   RZ(omega, qubit = target_qubit)])
        
#         list_gates.extend(qmachine.compiler.quil_to_native_quil(draft_circuit))
    
#     list_gates = [ ins for ins in list_gates if isinstance(ins, Gate)]
#     list_gates.extend( get_inverse_circuit(qmachine, list_gates) )
#     return list_gates

In [7]:
# def used_qubits_index(gates_sequence):
#     qubits = np.array([np.array(gate.qubits) for gate in gates_sequence])
#     qubits = np.array([ ele.index for sub_arr in qubits for ele in sub_arr]) #some gates might have multiple indices
#     qubits_indices = np.unique(qubits)
#     qubits_indices.sort()
#     return [ int(x) for x in qubits_indices ]

# def convert_measured_to_response_matrix(measured_outcome):
#     return 1 - np.bool_(np.sum(measured_outcome, axis = 1)) # 1 if it is equal to n_zero state

# def run_bench_experiment(qmachine, program, number_of_shots):
    
#     program = program.wrap_in_numshots_loop(number_of_shots)
    
#     #Run the program
#     executable = qmachine.compile(program)
#     result = qmachine.run(executable)
#     measured_outcome = result.readout_data.get('ro')
#     return measured_outcome

# def get_inverse_circuit(qmachine, gates_sequence):
#     """
#     :params gates_sequence: iterable sequence of circuit gates.
#     :return: numpy array of gates constructing inverse circuit of the input 
#     """
#     target_qubits = used_qubits_index(gates_sequence)
#     n_qubits = len(target_qubits)
    
#     prog = Program()
#     for gate in reversed(gates_sequence):
#         prog += daggered_gate(gate)
#     prog_daggered_native = qmachine.compiler.quil_to_native_quil(prog)
#     instructions = prog_daggered_native.instructions
#     inverting_gates_list = [ ins for ins in instructions if isinstance(ins, Gate)]
#     return np.array(inverting_gates_list)

# def generate_experiments(qmachine, target_qubits:list, circuit_gen_func, layers_num:int, exp_num:int):
#     n_qubits = len(target_qubits)
#     return np.array([circuit_gen_func(qmachine, target_qubits, layers_num) for i in range(exp_num)])

# def find_machine_response(qmachine, rb_experiments, number_of_shots):
#     """
#     It samples and record the accept or reject of the machine with given gate sequences
#     :return: response_matrix including accepts and rejects in columns
#     """
#     target_qubits = used_qubits_index(rb_experiments[0])
#     n_qubits = len(target_qubits)
#     sequ_num = len(rb_experiments)
#     response_matrix = np.zeros((sequ_num, number_of_shots))

#     for i_sequ, sequ in enumerate(tqdm(rb_experiments)):
#         prog = Program() #All qubits begin with |0> state
#         for gate in sequ:
#             prog += gate
        
#         #Do not let the quilc to alter the gates by optimization
#         prog = Program('PRAGMA PRESERVE_BLOCK') + prog
#         prog += Program('PRAGMA END_PRESERVE_BLOCK')
        
#         #Measurments
#         ro = prog.declare('ro', 'BIT', n_qubits)
#         for ind, qubit_ind in enumerate(target_qubits):
#             prog += MEASURE(qubit_ind, ro[ind])
            
#         response = convert_measured_to_response_matrix( run_bench_experiment(qmachine, prog, number_of_shots) )
#         response_matrix[i_sequ,:] = np.copy(response)
#     return response_matrix

## Single Qubit

In [8]:
qc = get_qc("2q-qvm")
# qc = get_qc("9q-square-noisy-qvm")
epsilon = 0.01
p_xi = epsilon/4
qc.qam.gate_noise=(p_xi,p_xi,p_xi)

In [8]:
# while 0 not in response_matrix:
exps = generate_experiments(qmachine = qc, target_qubits = [0], circuit_gen_func=native_rigetti_single_qubit_packs_generator, layers_num=10, exp_num=10)
response_matrix = find_machine_response(qc, exps, 10)
response_matrix

  0%|          | 0/10 [00:00<?, ?it/s]

array([[1., 1., 1., 1., 0., 1., 1., 1., 1., 1.],
       [1., 0., 0., 1., 1., 1., 0., 1., 1., 1.],
       [0., 1., 1., 1., 1., 1., 1., 0., 1., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 0., 0., 1., 1., 0., 1., 1., 0.],
       [1., 0., 1., 1., 1., 1., 1., 0., 0., 1.],
       [1., 1., 1., 1., 1., 0., 0., 1., 1., 1.],
       [0., 0., 1., 0., 1., 1., 0., 1., 0., 1.],
       [1., 1., 1., 1., 1., 1., 1., 0., 1., 1.],
       [1., 1., 0., 1., 1., 1., 1., 1., 1., 0.]])

## Two-qubits

In [10]:
qc = get_qc("2q-qvm")
# qc = get_qc("9q-square-noisy-qvm")
epsilon = 0.0
p_xi = epsilon/4
qc.qam.gate_noise=(p_xi,p_xi,p_xi)

# while 0 not in response_matrix:
exps = generate_experiments(qmachine = qc, target_qubits = [0,1], circuit_gen_func=native_universal_two_qubits_packs_generator, layers_num=1, exp_num=10)
response_matrix = find_machine_response(qc, exps, 10)
response_matrix

  0%|          | 0/10 [00:00<?, ?it/s]

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [9]:
draft_circuit = Program( [RZ(1, qubit = 0),
                          RY(2, qubit = 0),
                          RZ(3, qubit = 0)])

In [19]:
qc.compiler.quil_to_native_quil(draft_circuit).instructions

[<Gate RZ(-pi/2) 0>,
 <Gate RZ(-pi/2) 1>,
 <Gate CZ 1 0>,

In [20]:
draft_circuit = Program( CPHASE01(-np.pi/2, control=0, target=1),
                        CPHASE10(-np.pi/2, control=0, target=1) )
dc = qc.compiler.quil_to_native_quil(draft_circuit)

In [21]:
qc.compiler.native_quil_to_executable(dc).instructions

[<Gate RZ(-3*pi/2) 0>,
 <Gate RZ(-3*pi/2) 1>,
 <Gate CZ 1 0>,
 <Gate RZ(-pi) 0>,
 <Gate RZ(-pi) 1>,